# Categorizing Coding Languages with README Files
Mathias Boissevain | Stephen Kipkurui | Braeden Wright | Craig Calzado 

May 17, 2022

# Project Goal

**For this project we are building a model that can predict the main programming language of a repository, given the text of the README file. We will use 100 different README files from repositories available on GitHub, and use Natural Language Processing on the text within to build our model.**

__________

# Executive Summary

___________
***We explored the data and discovered:***

- 
- 
- 

***Recommandations:***
- 
- 
- 
- 

***Next Step:***
- 
- 
-


# Planning
- Setup repository environment (.gitignore, README.md)
- __Acquire Data:__
    - Import base_acquire.py function that scraps slightly over 100 Github repositories and returns raw data. 
- __Prepare Data:__
    - Perform basic clean of data
    - Tokenize data
    - Perform data stemming
    - Lemmatize the data
    - Removes Stopwords
    - Split the data into train, validate and test
       
- __Explore Data:__
    - Using Natural Language Processing (NLP) methodologies to further investigate the project goals.
    - Create Visualizations
    - Create and test Models 
    - Summarize findings
    
- __Deliverables:__
    - This Github repository
    - Powerpoint slides <a>https://docs.google.com/presentation/d/1uqp20l36bEUZK94Rtpab25Aq80QhSghQ1nXXIlHlKcU/edit#slide=id.p</a>
    - Stakeholders --> anyone/ public

## Import Menagerie

In [8]:
# Basics
import numpy as np
import pandas as pd
import nltk
import re
import os
import json

# Visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Speciality
from typing import Dict, List, Optional, Union, cast
import requests
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer


# Custom
import base_acquire
import wrangle

# Make sure we can see the full scale of the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# plotting defaults
plt.rc('figure', figsize=(11, 5))
plt.style.use('seaborn-whitegrid')
plt.rc('font', size=16)

# Acquire the data

**We acquired the data from repositories found on GitHub, and pulled them into a dataframe using web scrapping and functions from our acquire.py file and saved the data in a .json file**

In [ ]:
# Pull dataframe

# Data Wrangle

* tbd
* tbd                     
* Download cleaned data to a .json

# Data Exploration 

### Initial Questions
- What are the most common words in READMEs?

- Does the length of the README vary by programming language?

- Do different programming languages use a different number of unique words?

- Are there any words that uniquely identify a programming language?
--------
--------
## What are the most common words in READMEs?

**Takeaways:**
* tbd
-----------
## Does the length of the README vary by programming language?

**Takeaways:**
* tbd
-----------
## Do different programming languages use a different number of unique words?

**Takeaways:**
* tbd
-----------
## Are there any words that uniquely identify a programming language?

**Takeaways:**
* tbd
-----------
# Conclusion 

# Key Findings From Exploration

# Recommendation

* tbd
* tbd
* tbd

# Next Step

* TBD
* TBD
